In [ ]:
import torch
import glob
import numpy as np
import imageio
import cv2

import torch.utils.data as data
import torch.nn.functional as F
from torchvision.transforms import Compose

In [ ]:
class BadPixelMetric:
    def __init__(self, threshold=1.25, depth_cap=10):
        self.__threshold = threshold
        self.__depth_cap = depth_cap

    def compute_scale_and_shift(self, prediction, target, mask):
        # system matrix: A = [[a_00, a_01], [a_10, a_11]]
        a_00 = torch.sum(mask * prediction * prediction, (1, 2))
        a_01 = torch.sum(mask * prediction, (1, 2))
        a_11 = torch.sum(mask, (1, 2))

        # right hand side: b = [b_0, b_1]
        b_0 = torch.sum(mask * prediction * target, (1, 2))
        b_1 = torch.sum(mask * target, (1, 2))

        # solution: x = A^-1 . b = [[a_11, -a_01], [-a_10, a_00]] / (a_00 * a_11 - a_01 * a_10) . b
        x_0 = torch.zeros_like(b_0)
        x_1 = torch.zeros_like(b_1)

        det = a_00 * a_11 - a_01 * a_01
        # A needs to be a positive definite matrix.
        valid = det > 0

        x_0[valid] = (a_11[valid] * b_0[valid] - a_01[valid] * b_1[valid]) / det[valid]
        x_1[valid] = (-a_01[valid] * b_0[valid] + a_00[valid] * b_1[valid]) / det[valid]

        return x_0, x_1

    def __call__(self, prediction, target, mask):
        # transform predicted disparity to aligned depth
        target_disparity = torch.zeros_like(target)
        target_disparity[mask == 1] = 1.0 / target[mask == 1]

        scale, shift = self.compute_scale_and_shift(prediction, target_disparity, mask)
        prediction_aligned = scale.view(-1, 1, 1) * prediction + shift.view(-1, 1, 1)

        disparity_cap = 1.0 / self.__depth_cap
        prediction_aligned[prediction_aligned < disparity_cap] = disparity_cap

        prediciton_depth = 1.0 / prediction_aligned

        # bad pixel
        err = torch.zeros_like(prediciton_depth, dtype=torch.float)

        err[mask == 1] = torch.max(
            prediciton_depth[mask == 1] / target[mask == 1],
            target[mask == 1] / prediciton_depth[mask == 1],
        )

        err[mask == 1] = (err[mask == 1] > self.__threshold).float()

        p = torch.sum(err, (1, 2)) / torch.sum(mask, (1, 2))

        return 100 * torch.mean(p)


In [1]:
import os

def count_images(directory, extensions={".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff", ".webp"}):
    image_count = 0
    
    for root, _, files in os.walk(directory):
        image_count += sum(1 for file in files if os.path.splitext(file)[1].lower() in extensions)
    
    return image_count



In [23]:
# Example usage
directory_path = "/home/gkmo/workspace/data/KITTI_eigen_split/train/rgb"  # Change this to the folder you want to scan
print(f"Total image files: {count_images(directory_path)/2}")

Total image files: 11579.0


In [26]:
# Example usage
directory_path = "/home/gkmo/workspace/data/KITTI_eigen_split/test/output_rgb"  # Change this to the folder you want to scan
print(f"Total image files: {count_images(directory_path)}")

Total image files: 652


In [16]:
import os
import shutil

def organize_images(train_root, output_rgb="output_rgb", output_depth="output_depth"):
    # Paths for output folders
    rgb_output_dir = os.path.join(train_root, output_rgb)
    depth_output_dir = os.path.join(train_root, output_depth)
    os.makedirs(rgb_output_dir, exist_ok=True)
    os.makedirs(depth_output_dir, exist_ok=True)

    skipped_images = []  # List to store skipped images

    # Traverse all date/sequence folders
    for root, _, files in os.walk(train_root):
        if "image_02/data" in root:
            for file in files:
                if file.endswith(".png"):
                    # Construct the corresponding depth path
                    depth_root = root.replace("image_02/data", "proj_depth/groundtruth/image_02")
                    src_depth = os.path.join(depth_root, file)

                    # Check if the depth image exists
                    if os.path.exists(src_depth):
                        # Get the relative path (excluding 'train_root')
                        relative_path = os.path.relpath(root, train_root).replace(os.sep, "_")
                        new_filename = f"{relative_path}_{file}"  # e.g., "date1_sequence1_image_02_data_000001.png"

                        # Copy RGB image
                        src_rgb = os.path.join(root, file)
                        dst_rgb = os.path.join(rgb_output_dir, new_filename)
                        shutil.copy2(src_rgb, dst_rgb)

                        # Copy Depth image
                        dst_depth = os.path.join(depth_output_dir, new_filename)
                        shutil.copy2(src_depth, dst_depth)
                    else:
                        # If depth image is missing, log the skipped file
                        skipped_images.append(os.path.join(root, file))

    print(f"Images copied into {rgb_output_dir} and {depth_output_dir}, ensuring all pairs match.")

    # Print skipped images
    if skipped_images:
        print("\nSkipped the following RGB images due to missing depth maps:")
        for img in skipped_images:
            print(img)
    else:
        print("\nAll RGB images had corresponding depth maps.")

In [18]:
# Example usage
train_directory = "/home/gkmo/Downloads/archive/train"  # Change this to your actual train folder
organize_images(train_directory)


Images copied into /home/gkmo/Downloads/archive/train/output_rgb and /home/gkmo/Downloads/archive/train/output_depth, ensuring all pairs match.

All RGB images had corresponding depth maps.
